In [57]:
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
%matplotlib qt5
eps = np.finfo(float).eps

In [2]:
from posarmctools.readdata import *
%load_ext autoreload
%autoreload 2

In [3]:
import sys

In [4]:
sys.path.insert(0, "/home/pleroy/DEV/processing/focalization_python")
from posarutils.other.read_data_and_build_rd import *
from posarutils.other.PosarMCParameters import *

In [5]:
data_date = "2017_10_12_11_52_43"
data_dir = "/home/pleroy/DATA/PoSAR-MC_PIMA_TEST-3/2017_10_12/" + data_date

In [6]:
params_filename = data_dir + "/" + data_date + "_parameters.xml"
params = PosarMCParameters( params_filename )

In [81]:
blocksPerFile = params.blocksPerFile
samplesPerRamp = params.samplesPerRamp
rampsPerFile = params.rampsPerFile
samplesPerFile = samplesPerRamp * rampsPerFile

print( "blocksPerFile = {}, samplesPerRamp = {}, rampsPerFile = {}".format(
    blocksPerFile, samplesPerRamp, rampsPerFile )
     )

# selection in record
firstFile = 0
nbFiles = 100
lastFile = firstFile + nbFiles - 1
firstBlock = firstFile * blocksPerFile

nbPos = nbFiles * rampsPerFile

blocksPerFile = 6, samplesPerRamp = 6000, rampsPerFile = 1500


### Read the data

In [8]:
timeSerie_A = np.zeros( samplesPerFile )
timeSerie_B = np.zeros( samplesPerFile )

adc_A = np.zeros( (nbFiles, samplesPerFile) )
adc_B = np.zeros( (nbFiles, samplesPerFile) )

In [9]:
lastBlock = (nbFiles-1) * blocksPerFile + firstBlock
for k in range( nbFiles ):
    
    # read the data
    nb = str( int( k * blocksPerFile + firstBlock) )
    filename = data_dir + "/record" + nb + ".bin"
    readFile( filename, samplesPerFile, timeSerie_A, timeSerie_B )
    
    print("k = {}, block = {} / {}".format(k, nb, lastBlock ))
    
    adc_A[ k, : ] = timeSerie_A
    adc_B[ k, : ] = timeSerie_B

k = 0, block = 0 / 594
k = 1, block = 6 / 594
k = 2, block = 12 / 594
k = 3, block = 18 / 594
k = 4, block = 24 / 594
k = 5, block = 30 / 594
k = 6, block = 36 / 594
k = 7, block = 42 / 594
k = 8, block = 48 / 594
k = 9, block = 54 / 594
k = 10, block = 60 / 594
k = 11, block = 66 / 594
k = 12, block = 72 / 594
k = 13, block = 78 / 594
k = 14, block = 84 / 594
k = 15, block = 90 / 594
k = 16, block = 96 / 594
k = 17, block = 102 / 594
k = 18, block = 108 / 594
k = 19, block = 114 / 594
k = 20, block = 120 / 594
k = 21, block = 126 / 594
k = 22, block = 132 / 594
k = 23, block = 138 / 594
k = 24, block = 144 / 594
k = 25, block = 150 / 594
k = 26, block = 156 / 594
k = 27, block = 162 / 594
k = 28, block = 168 / 594
k = 29, block = 174 / 594
k = 30, block = 180 / 594
k = 31, block = 186 / 594
k = 32, block = 192 / 594
k = 33, block = 198 / 594
k = 34, block = 204 / 594
k = 35, block = 210 / 594
k = 36, block = 216 / 594
k = 37, block = 222 / 594
k = 38, block = 228 / 594
k = 39, block =

In [47]:
# reRead one file
filename = data_dir + "/record0.bin"
readFile( filename, samplesPerFile, timeSerie_A, timeSerie_B )
adc_A[ 0, : ] = timeSerie_A
adc_B[ 0, : ] = timeSerie_B

In [45]:
A_reshaped = adc_A.reshape(nbPos, samplesPerRamp)
B_reshaped = adc_B.reshape(nbPos, samplesPerRamp)

In [80]:
A_reshaped.shape

(150000, 6000)

**Check that the first ramp is an up ramp**

In [48]:
plt.figure()
plt.plot( A_reshaped[0, :] )
plt.title("first acquisition\n" + data_date)
plt.grid()

Work a little bit on the data

In [85]:
plt.figure()
nbRampsToPlot = 10
step = int( nbPos / nbRampsToPlot )
for k in range(nbRampsToPlot):
    pos = k * step
    plt.plot( 20 * np.log10( np.abs( np.fft.ifft( A_reshaped[ pos, 0:3000 ] ) + eps ) ), label="{}".format(pos) )
plt.legend()
plt.grid()

In [77]:
A0 = np.zeros(samplesPerRamp)
A0 = A_reshaped[0, :]
nbPointsToRemove = 5

fftA0 = np.abs(np.fft.fft(A0))

fftA0_b = np.abs(np.fft.fft(A0))
fftA0_b[0:nbPointsToRemove] = eps
fftA0_b[-nbPointsToRemove:] = eps
A0_b = np.real(np.fft.ifft( fftA0_b ))

sa_A0 = signal.hilbert( A0 )
fft_sa_A0 = np.fft.fft( sa_A0 )
# remove bins from analytic signal
fft_sa_A0[333:342] = 0
fft_sa_A0[2690:2693] = 0
sa_A0_cleaned = np.fft.ifft( fft_sa_A0 )

In [78]:
plt.figure()
plt.plot(20*np.log10(np.abs(fftA0)), label="fftA0")
plt.plot(20*np.log10(np.abs(fftA0_b)), label="fftA0_b")
plt.plot(20*np.log10(np.abs(fft_sa_A0)), label="fft_sa_A0")
plt.grid()
plt.legend()

/opt/Py3-6-Venv/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log10
  after removing the cwd from sys.path.


In [79]:
plt.figure()
plt.plot(A0, label="A0")
plt.plot(A0_b, label="A0_b")
plt.plot(np.real(sa_A0_cleaned), label="sa_A0_cleaned")
plt.grid()
plt.legend()

**Set the shifted flag properly**

In [ ]:
shifted = 1

In [ ]:
ifft_a = np.fft.ifft( (A_reshaped[::,0:3000]), axis = 1 )

In [ ]:
plt.figure()
vmin = -15
vmax = 0
nbPointsInRange = 500
plt.plot(  20 * np.log10( np.abs( ifft_a[2500,:] ) ) )
plt.plot( np.ones( nbPointsInRange ) * vmin, "b" )
plt.plot( np.ones( nbPointsInRange ) * vmax, "r" )

In [ ]:
plt.matshow( 20 * np.log10( np.abs( ifft_a[:,:] ) ), 
            aspect='auto', vmin=-15, vmax=0, cmap='jet' )
plt.title("Track 1 - record 2017_10_12_11_47_05\nFiles {} to {}".format(firstFile, lastFile) )

## Remove coupling from raw data

In [ ]:
coupling_A = np.average( A_reshaped, 0 )
A2 = A_reshaped - coupling_A

## Build RD

In [ ]:
RDc = build_rd_from_data( params, A_reshaped, shifted )

In [ ]:
coupling_RD = np.average(RDc, 0)
np.save( data_dir + "/coupling_RD_files_{}_{}".format(firstFile, lastFile), coupling_RD )

In [ ]:
RD2 = RDc - coupling_RD

In [ ]:
plt.figure()
plt.plot(np.abs(coupling_RD), 'k', label='coupling')
plt.legend()
plt.grid()

In [ ]:
np.save(data_dir + '/RD_files_{}_{}'.format(firstFile, lastFile), RDc)

In [ ]:
ifft_RD = np.fft.ifft(RDc, axis = 1)
print('ifft_RD done')

In [ ]:
nbPointsInRange = 300
x0 = 0
x1 = x0 + nbPointsInRange
y0 = 0
y1 = 0
extent = [ x0, x1, y0, y1 ]
rangeStart = 0
rangeStop = 500
azStart = 6000
azStop = 15000

In [ ]:
plt.matshow( 20 * np.log10( np.abs( ifft_RD[:, rangeStart:rangeStop] ) ), aspect='auto')
plt.title(data_date)
ax = plt.gca()
plt.colorbar()

In [ ]:
fft2d = np.fft.fftshift( np.fft.fft( ifft_RD, axis = 0 ) , axes = 0)
print('fft2d done')

In [ ]:
plt.matshow( 20 * np.log( np.abs( fft2d[:, rangeStart:rangeStop] ) ), aspect='auto', cmap='jet', vmin=-60, vmax=50)
plt.title(data_date)
ax = plt.gca()
plt.colorbar()

In [ ]:
adc_A_all = adc_A.reshape( rampsPerFile * nbFiles * samplesPerRamp )

In [ ]:
plt.figure()
ramp = 10
shift = 38
sampleStart = ramp * samplesPerRamp
plt.plot( adc_A_all[sampleStart : sampleStart+6000 ], 'k',
         label='both ramps' )
plt.plot( adc_A_all[sampleStart : sampleStart+3000 ],
         label='up ramp' )
plt.plot( np.flipud( adc_A_all[ sampleStart+3000 : sampleStart+6000 ] ),
         label='flipped down ramp' )
plt.plot( np.flipud( adc_A_all[ (sampleStart+3000+shift) : (sampleStart+6000+shift) ] ),
         label='flipped down ramp SHIFTED' )
plt.grid()
plt.legend()

In [ ]:
plt.figure()
ramp = 0
plt.plot( A2[ramp, 0 : 6000 ], 'k', label='both ramps' )
plt.plot( A2[ramp, 0 : 3000 ], label='up ramp' )
plt.plot( np.flipud( A2[ramp, 3000 : 6000 ] ), label='flipped down ramp' )
plt.grid()
plt.legend()
plt.title('A2')

In [ ]:
params.samplesPerRamp

In [ ]:
shifted

In [ ]:
import scipy.signal as signal

In [ ]:
s1 = A_reshaped[ramp, 0 : 3000 ]
s2 = np.flipud( A_reshaped[ramp, 4000 : 5500 ] )

corr = signal.correlate( s1, s2, mode='valid' )

plt.figure()

plt.subplot(211)
plt.plot( corr, '.' )
plt.grid()

plt.subplot(212)
plt.plot( s1 )
plt.plot( s2 )
plt.plot( np.arange(1500) + 538, s2 )
plt.grid()

plt.suptitle( 'max = {}'.format( np.where( corr == np.max(corr) )[0] ) )

In [ ]:
plt.plot(s1)